In [2]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager

# Set Chrome options
chrome_options = Options()
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--start-maximized")

# Initialize WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

def scroll_to_bottom(driver):
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1.5)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

def scrape_culture_circle_product(url):
    driver.get(url)
    scroll_to_bottom(driver)

    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, "a_productHeading__jLymj"))
    )

    try:
        read_more = driver.find_element(By.XPATH, "//button[contains(text(), 'Read more')]")
        driver.execute_script("arguments[0].click();", read_more)
        time.sleep(1)
    except:
        pass

    soup = BeautifulSoup(driver.page_source, 'html.parser')

    title_div = soup.find("div", class_="a_productHeading__jLymj")
    product_name = title_div.find("h2").text.strip() if title_div and title_div.find("h2") else ""
    title = f"{product_name}"

    desc_tag = soup.find("p", class_="w-full")
    description = desc_tag.get_text(strip=True) if desc_tag else "N/A"

    image_tags = soup.find_all("img", class_="a_thumbnailImage___06oR")
    images = [img['src'] for img in image_tags if img.get('src')]

    # Sizes and Prices (remove duplicates using a set of tuples)
    sizes_prices = []
    seen = set()
    size_wrappers = soup.find_all("div", class_="a_sizeSlide__FHiSL")
    for size_div in size_wrappers:
        size = size_div.find("div", class_="a_sizeSlideSize__jBG1p")
        price = size_div.find("p", class_="a_sizeSlidePrice__NASxX")
        if size and price:
            size_text = size.get_text(strip=True)
            price_text = price.get_text(strip=True)
            if (size_text, price_text) not in seen:
                seen.add((size_text, price_text))
                sizes_prices.append({
                    "size": size_text,
                    "price": price_text
                })

    return {
        "title": title,
        "url": url,
        "description": description,
        "images": images,
        "sizes_prices": sizes_prices
    }

# --- USER INPUT ---
url = input("Enter the culture-circle product URL: ").strip()

# --- SCRAPE ---
product_data = scrape_culture_circle_product(url)

# --- DISPLAY ---
print("\n--- SCRAPED PRODUCT DATA ---")
print(f"Title       : {product_data['title']}")
print(f"URL         : {product_data['url']}")
print(f"Description : {product_data['description']}")
print("\nImages:")
for img in product_data["images"]:
    print(f" - {img}")
print("\nSize-Price Mapping :")
for size_price in product_data["sizes_prices"]:
    print(f" - {size_price['size']} : {size_price['price']}")

# --- CREATE DATAFRAME ---
rows = []
for size_price in product_data['sizes_prices']:
    rows.append({
        "title": product_data['title'],
        "url": product_data['url'],
        "size": size_price['size'],
        "price": size_price['price'],
        "description": product_data['description'],
        "images": ", ".join(product_data['images']),  # convert list to string
    })

df = pd.DataFrame(rows)

# --- Save to CSV if needed ---
# df.to_csv("scraped_product_data.csv", index=False)

# --- Close Browser ---
driver.quit()


Enter the culture-circle product URL:  https://www.culture-circle.com/products/all/adidas-yeezy-foam-rnr-carbon



--- SCRAPED PRODUCT DATA ---
Title       : Adidas Yeezy Foam Rnr Carbon
URL         : https://www.culture-circle.com/products/all/adidas-yeezy-foam-rnr-carbon
Description : TheAdidas Yeezy Foam RNR Carbonis not just a sneaker; it’s a statement. Designed by Kanye West, this model embodies a futuristic vision that has captivated sneaker enthusiasts. With its slip-on style and dark, muted tones, it seamlessly blends into both urban landscapes and casual settings, making it a versatile choice for any occasion.

Crafted from a unique combination ofEVA foamand algae, the Foam RNR Carbon prioritizes sustainability without compromising on comfort. The design features ventilation holes that enhance breathability, ensuring your feet stay cool during those long summer strolls. The outsole is engineered for grip, providing stability whether you're navigating city streets or enjoying a leisurely day outdoors.

When it comes to fit, the Foam RNR Carbon is available in a range of sizes for men, wome

In [3]:
df

,title,url,size,price,description,images
0,Adidas Yeezy Foam Rnr Carbon,https://www.culture-circle.com/products/all/ad...,UK2.5,"₹5,038",TheAdidas Yeezy Foam RNR Carbonis not just a s...,https://culture-cirlce-static-dge7g8b8eue6fvcv...
1,Adidas Yeezy Foam Rnr Carbon,https://www.culture-circle.com/products/all/ad...,UK4,"₹5,598",TheAdidas Yeezy Foam RNR Carbonis not just a s...,https://culture-cirlce-static-dge7g8b8eue6fvcv...
2,Adidas Yeezy Foam Rnr Carbon,https://www.culture-circle.com/products/all/ad...,UK5,"₹6,600",TheAdidas Yeezy Foam RNR Carbonis not just a s...,https://culture-cirlce-static-dge7g8b8eue6fvcv...
3,Adidas Yeezy Foam Rnr Carbon,https://www.culture-circle.com/products/all/ad...,UK6(EU40),"₹9,674",TheAdidas Yeezy Foam RNR Carbonis not just a s...,https://culture-cirlce-static-dge7g8b8eue6fvcv...
4,Adidas Yeezy Foam Rnr Carbon,https://www.culture-circle.com/products/all/ad...,UK6(EU39),"₹7,975",TheAdidas Yeezy Foam RNR Carbonis not just a s...,https://culture-cirlce-static-dge7g8b8eue6fvcv...
5,Adidas Yeezy Foam Rnr Carbon,https://www.culture-circle.com/products/all/ad...,UK6,"₹6,652",TheAdidas Yeezy Foam RNR Carbonis not just a s...,https://culture-cirlce-static-dge7g8b8eue6fvcv...
6,Adidas Yeezy Foam Rnr Carbon,https://www.culture-circle.com/products/all/ad...,UK7,"₹6,652",TheAdidas Yeezy Foam RNR Carbonis not just a s...,https://culture-cirlce-static-dge7g8b8eue6fvcv...
7,Adidas Yeezy Foam Rnr Carbon,https://www.culture-circle.com/products/all/ad...,UK8,"₹8,050",TheAdidas Yeezy Foam RNR Carbonis not just a s...,https://culture-cirlce-static-dge7g8b8eue6fvcv...
8,Adidas Yeezy Foam Rnr Carbon,https://www.culture-circle.com/products/all/ad...,UK9,"₹7,882",TheAdidas Yeezy Foam RNR Carbonis not just a s...,https://culture-cirlce-static-dge7g8b8eue6fvcv...
9,Adidas Yeezy Foam Rnr Carbon,https://www.culture-circle.com/products/all/ad...,UK10,"₹8,798",TheAdidas Yeezy Foam RNR Carbonis not just a s...,https://culture-cirlce-static-dge7g8b8eue6fvcv...


In [2]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager

# Set Chrome options
chrome_options = Options()
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--start-maximized")

# Initialize WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

def scroll_to_bottom(driver):
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1.5)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

def scrape_culture_circle_product(url):
    driver.get(url)
    scroll_to_bottom(driver)

    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, "a_productHeading__jLymj"))
    )

    try:
        read_more = driver.find_element(By.XPATH, "//button[contains(text(), 'Read more')]")
        driver.execute_script("arguments[0].click();", read_more)
        time.sleep(1)
    except:
        pass

    soup = BeautifulSoup(driver.page_source, 'html.parser')

    title_div = soup.find("div", class_="a_productHeading__jLymj")
    product_name = title_div.find("h2").text.strip() if title_div and title_div.find("h2") else ""
    title = f"{product_name}"

    desc_tag = soup.find("p", class_="w-full")
    description = desc_tag.get_text(strip=True) if desc_tag else "N/A"

    image_tags = soup.find_all("img", class_="a_thumbnailImage___06oR")
    images = [img['src'] for img in image_tags if img.get('src')]

    # Sizes and Prices (remove duplicates using a set of tuples)
    sizes_prices = []
    seen = set()
    size_wrappers = soup.find_all("div", class_="a_sizeSlide__FHiSL")
    for size_div in size_wrappers:
        size = size_div.find("div", class_="a_sizeSlideSize__jBG1p")
        price = size_div.find("p", class_="a_sizeSlidePrice__NASxX")
        if size and price:
            size_text = size.get_text(strip=True)
            price_text = price.get_text(strip=True)
            if (size_text, price_text) not in seen:
                seen.add((size_text, price_text))
                sizes_prices.append({
                    "size": size_text,
                    "price": price_text
                })

    return {
        "title": title,
        "url": url,
        "description": description,
        "images": images,
        "sizes_prices": sizes_prices
    }

# --- USER INPUT ---
url = input("Enter the culture-circle product URL: ").strip()

# --- SCRAPE ---
product_data = scrape_culture_circle_product(url)

# --- DISPLAY ---
print("\n--- SCRAPED PRODUCT DATA ---")
print(f"Title       : {product_data['title']}")
print(f"URL         : {product_data['url']}")
print(f"Description : {product_data['description']}")
print("\nImages:")
for img in product_data["images"]:
    print(f" - {img}")
print("\nSize-Price Mapping :")
for size_price in product_data["sizes_prices"]:
    print(f" - {size_price['size']} : {size_price['price']}")

# --- CREATE DATAFRAME ---
rows = []
for size_price in product_data['sizes_prices']:
    rows.append({
        "title": product_data['title'],
        "url": product_data['url'],
        "size": size_price['size'],
        "price": size_price['price'],
        "description": product_data['description'],
        "sku":"N/A",
        "images": ", ".join(product_data['images']),  # convert list to string
    })

df = pd.DataFrame(rows)

# --- Save to CSV if needed ---
# df.to_csv("scraped_product_data.csv", index=False)

# --- Close Browser ---
driver.quit()


Enter the culture-circle product URL:  https://www.culture-circle.com/products/all/adidas-yeezy-foam-rnr-carbon



--- SCRAPED PRODUCT DATA ---
Title       : Adidas Yeezy Foam Rnr Carbon
URL         : https://www.culture-circle.com/products/all/adidas-yeezy-foam-rnr-carbon
Description : TheAdidas Yeezy Foam RNR Carbonis not just a sneaker; it’s a statement. Designed by Kanye West, this model embodies a futuristic vision that has captivated sneaker enthusiasts. With its slip-on style and dark, muted tones, it seamlessly blends into both urban landscapes and casual settings, making it a versatile choice for any occasion.

Crafted from a unique combination ofEVA foamand algae, the Foam RNR Carbon prioritizes sustainability without compromising on comfort. The design features ventilation holes that enhance breathability, ensuring your feet stay cool during those long summer strolls. The outsole is engineered for grip, providing stability whether you're navigating city streets or enjoying a leisurely day outdoors.

When it comes to fit, the Foam RNR Carbon is available in a range of sizes for men, wome

In [3]:
df

,title,url,size,price,description,sku,images
0,Adidas Yeezy Foam Rnr Carbon,https://www.culture-circle.com/products/all/ad...,UK2.5,"₹5,038",TheAdidas Yeezy Foam RNR Carbonis not just a s...,N/A,https://culture-cirlce-static-dge7g8b8eue6fvcv...
1,Adidas Yeezy Foam Rnr Carbon,https://www.culture-circle.com/products/all/ad...,UK4,"₹5,598",TheAdidas Yeezy Foam RNR Carbonis not just a s...,N/A,https://culture-cirlce-static-dge7g8b8eue6fvcv...
2,Adidas Yeezy Foam Rnr Carbon,https://www.culture-circle.com/products/all/ad...,UK5,"₹6,600",TheAdidas Yeezy Foam RNR Carbonis not just a s...,N/A,https://culture-cirlce-static-dge7g8b8eue6fvcv...
3,Adidas Yeezy Foam Rnr Carbon,https://www.culture-circle.com/products/all/ad...,UK6(EU40),"₹9,674",TheAdidas Yeezy Foam RNR Carbonis not just a s...,N/A,https://culture-cirlce-static-dge7g8b8eue6fvcv...
4,Adidas Yeezy Foam Rnr Carbon,https://www.culture-circle.com/products/all/ad...,UK6(EU39),"₹7,975",TheAdidas Yeezy Foam RNR Carbonis not just a s...,N/A,https://culture-cirlce-static-dge7g8b8eue6fvcv...
5,Adidas Yeezy Foam Rnr Carbon,https://www.culture-circle.com/products/all/ad...,UK6,"₹6,652",TheAdidas Yeezy Foam RNR Carbonis not just a s...,N/A,https://culture-cirlce-static-dge7g8b8eue6fvcv...
6,Adidas Yeezy Foam Rnr Carbon,https://www.culture-circle.com/products/all/ad...,UK7,"₹6,652",TheAdidas Yeezy Foam RNR Carbonis not just a s...,N/A,https://culture-cirlce-static-dge7g8b8eue6fvcv...
7,Adidas Yeezy Foam Rnr Carbon,https://www.culture-circle.com/products/all/ad...,UK8,"₹8,050",TheAdidas Yeezy Foam RNR Carbonis not just a s...,N/A,https://culture-cirlce-static-dge7g8b8eue6fvcv...
8,Adidas Yeezy Foam Rnr Carbon,https://www.culture-circle.com/products/all/ad...,UK9,"₹7,882",TheAdidas Yeezy Foam RNR Carbonis not just a s...,N/A,https://culture-cirlce-static-dge7g8b8eue6fvcv...
9,Adidas Yeezy Foam Rnr Carbon,https://www.culture-circle.com/products/all/ad...,UK10,"₹8,798",TheAdidas Yeezy Foam RNR Carbonis not just a s...,N/A,https://culture-cirlce-static-dge7g8b8eue6fvcv...
